In [3]:
# 추천알고리즘(아이템기반)
# 코사인유사도

def predict_rating(ratings_array, item_similarity):
    import numpy as np
    sum_sr = ratings_array @ item_similarity
            # u x i , item_similarity = ixi
        # @ 연산자는 np.matmul, np.dot 행렬 곱을 해줌
    sum_abs = np.array([np.abs(item_similarity).sum(axis=1)])
    
    ratings_pred = sum_sr / sum_abs
    return ratings_pred

def recommend_place_by_memId(pred_df, memId, no_list, top_n=10):
    # 가보지 않은 여행지 리스트 중 예측 점수가 가장 높은 순서 .series
    recommend_place = pred_df.loc[memId, no_list].sort_values(ascending=False)[:top_n]
    return recommend_place

def no_place(placedata, memId):
    memId_rating = placedata.loc[memId,:]
    #user_id 별로 평점 정보
    no_place = memId_rating[memId_rating==0].index.tolist()
    # 보지않은 영화 리스트
    place_list = placedata.columns.tolist()
    # 모든 영화를 리스트 만들기
    no_list = [place for place in place_list if place in no_place]
    # 안 본 영화 리스트 
    return no_list

def load_data(sql):
    import cx_Oracle
    import pandas as pd

    # 1.오라클 주소
    dsn=cx_Oracle.makedsn("project-db-stu.ddns.net",1524,"xe")

    # 2.오라클 접속 유저 정보
    db = cx_Oracle.connect('campus_b_0825_1','smhrd1',dsn)

    # 3.결과 데이터를 담을 메모리 이름을 cursor로 선언
    cursor = db.cursor()

    # 4.작성한 SQL 쿼리문의 결과는 cursor 메모리에 담긴다
    cursor.execute(sql)         

    # 5.cursor 메모리에 담길 결과를 한 번에 row 변수에 담는다
    row = cursor.fetchall()
    mem_info = pd.DataFrame(row)

    # 6.컬럼명을 리스트에 담기
    colname = cursor.description
    col=[]
    for i in colname:
        col.append(i[0].lower())

    # 컬럼 리스트를 테이터에 결합시키기
    # list(row)는 열로, col은 행으로 표현
    loadData = pd.DataFrame(list(row),columns=col)

    cursor.close()
    db.close()

    return loadData

def rec_pl_mem(memId, place, review):
    import pandas as pd
    from sklearn.metrics.pairwise import cosine_similarity
    import json
    
    # 데이터 불러오기
    placeInfo = place
    reviewInfo = review

    # 아이템 기반 협업 필터링
    # 데이터 병합
    data = pd.merge(reviewInfo, placeInfo, on='place_no')
    column = ['mem_no', 'place_no', 'place_rating','place_name']
    data = data[column]

    # 사용자별 여행지 평점 
    # pivot_table(value, index, columns) 
    placedata = data.pivot_table('place_rating', 'mem_no', 'place_name')

    #NaN = 0 으로 채우기
    placedata.fillna(0, inplace=True)

    # 여행지별 사용자 평점 
    # index <-> columns 위치를 바꾸기
    # 아이템기반 협업 필터링을 하기위해 사용자-아이템 행렬을 아이템-사용자 행렬을 사용
    placedata_T = placedata.T

    from sklearn.metrics.pairwise import cosine_similarity

    # 코사인 유사도
    sklearn_cosine = cosine_similarity(placedata_T, placedata_T)
    item_similarity = pd.DataFrame(sklearn_cosine, index=placedata_T.index, columns = placedata_T.index)

    
    rating_pred = predict_rating(placedata.values, item_similarity.values)
    rating_pred_matrix = pd.DataFrame(data=rating_pred, index = placedata.index, columns = placedata.columns)

    # 로그인한 사용자의 No
    memId = memId

    # 가보지 않은 여행지 리스트
    no_place_data = no_place(placedata, memId)

    # 가보지 않은 여행지 리스트 중 예측 점수가 가장 높은 순서 .series
    recommend_place = recommend_place_by_memId(rating_pred_matrix, memId, no_place_data, top_n=10)

    # DataFrame -> json 형태로 변환
    recom_places = pd.DataFrame(data=recommend_place.values, index=recommend_place.index, columns=['pred_score'])
    
    return recom_places

In [4]:
# JavaScript <-> Python 서버통신

#flask == python 서버 구축 프레임워크
from flask import Flask, request, redirect, jsonify
from flask_cors import CORS

#flask 서버 객체 생성
# __name__ : 파일명을 담고있는 전역변수
app = Flask(__name__)

# Ajax : 동일출처(CORS) : 같은서버 내에서만 통신 가능
# 동일 출처 정책 완화
CORS(app)

# route : controller, 요청을 구분하는단위
# app.route("/urlmapping", methods=["Get? or Post?"])

@app.route("/json", methods=['GET', 'POST'])
def place_json():

    placeInfo = load_data("""select * from place_info""")
    reviewInfo = load_data("""select * from place_review_info""")
    
    memNo = int(request.form['memId'])
    request.args['memId']
    
    placeNo = rec_pl_mem(memNo, placeInfo, reviewInfo).index
    info = placeInfo[placeInfo["place_no"].isin(placeNo)]
    recom_places_json = info.to_json(force_ascii=False, orient="records", indent=4)

    return jsonify(recom_places_json)

# 서버 실행
if __name__ == '__main__':
    app.run(host="127.0.0.1", port=3300)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3300
Press CTRL+C to quit
[2022-09-30 11:24:29,433] ERROR in app: Exception on /json [POST]
Traceback (most recent call last):
  File "c:\Users\smhrd\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\smhrd\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\smhrd\AppData\Local\Programs\Python\Python310\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "c:\Users\smhrd\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\smhrd\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return